In [4]:
import pandas as pd
import json

en_de = json.load(open("multilingual_data/MLQA_V1/dev/dev-context-en-question-de.json"))
de_en = json.load(open("multilingual_data/MLQA_V1/dev/dev-context-de-question-en.json"))

paragraphs_en_de = {}
paragraphs_de_en = {}

for article in en_de["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            paragraphs_en_de[qa["id"]] = paragraph

for article in de_en["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            paragraphs_de_en[qa["id"]] = paragraph

reverse_en_de = {}
for key, value in paragraphs_en_de.items():
    if value["context"] not in reverse_en_de:
        reverse_en_de[value["context"]] = [key]
    else:
        reverse_en_de[value["context"]].append(key)

reverse_de_en = {}
for key, value in paragraphs_de_en.items():
    if value["context"] not in reverse_de_en:
        reverse_de_en[value["context"]] = [key]
    else:
        reverse_de_en[value["context"]].append(key)

paragraphs_en = []
paragraphs_de = []
for qa in paragraphs_en_de:
    if sorted(reverse_en_de[paragraphs_en_de[qa]["context"]]) != sorted(reverse_de_en[paragraphs_de_en[qa]["context"]]):
        continue
    if paragraphs_en_de[qa] not in paragraphs_en:
        paragraphs_en.append(paragraphs_en_de[qa])
        paragraphs_de.append(paragraphs_de_en[qa])


questions = []

for paragraph in paragraphs_en:
    for qa in paragraph["qas"]:
        questions.append({"id": qa["id"], "question_de": qa["question"], "answer_en": qa["answers"][0]["text"], "paragraph_en": paragraph["context"]})

df = pd.DataFrame(questions)

for paragraph in paragraphs_de:
    for qa in paragraph["qas"]:
        df.loc[df["id"] == qa["id"], "question_en"] = qa["question"]
        df.loc[df["id"] == qa["id"], "answer_de"] = qa["answers"][0]["text"]
        df.loc[df["id"] == qa["id"], "paragraph_de"] = paragraph["context"]

df.head()

,id,question_de,answer_en,paragraph_en,question_en,answer_de,paragraph_de
0,bc05f276f966ee2a28d92ad7da46ae82defb3c57,Wie viele Scanlinien hat NTSC als Kompromiss a...,525,The National Television System Committee was e...,How many scan lines did NTSC select as a compr...,525-zeiligen,Das National Television Systems Committee wurd...
1,04203425260fcd4cfddf76f0d7370073f8490912,Wie viele Zeilen waren es?,525,The actual figure of 525 lines was chosen as a...,How many lines was it?,525,"Der NTSC-Standard, oder korrekterweise M-Stand..."
2,aa6b395a5cb2ead221c88b62d2ce1a5ad0b9c498,Aus welchen Hauptelementen bestand nach Ansich...,"earth, fire, air and water","Around 420 BC, Empedocles stated that all matt...",What are the main elements that Empedocles cla...,"Erde, Wasser, Luft und Feuer.","Thales von Milet (Urstoff: Wasser), Anaximenes..."
3,2a0768a3f438fd000fea3c5945838de6711b7f58,Welche Stromquelle verwendet ein tragbarer Com...,AC power source,A portable computer (discontinued) is a genera...,What power source does a portable computer use?,Stromanschluss,"Tragbare Computer (eingestellt), ist ein einfa..."
4,21ec495b4c017f9189d6f8c3a2c88e46cd45977b,Worin besteht für manche Menschen die Schwieri...,Screens and keyboards tend to be small,Human interface with device: Screens and keybo...,what is it difficult for some people to use mo...,"Bildschirme und Tastaturen sind meist sehr klein,",Schnittstelle des Menschen zum Gerät: Bildschi...


In [5]:
df_2 = df[["question_en", "paragraph_en", "answer_en"]].copy()

df_2 = df_2.rename({"question_en": "Query", "paragraph_en": "Document", "answer_en": "Answer"}, axis=1)

df_2.to_csv("multilingual_data/mlqa_en-en.tsv", sep="\t", index=False)


df_2 = df[["question_de", "paragraph_en", "answer_de"]].copy()

df_2 = df_2.rename({"question_de": "Query", "paragraph_en": "Document", "answer_de": "Answer"}, axis=1)

df_2.to_csv("multilingual_data/mlqa_en-de.tsv", sep="\t", index=False)


df_2 = df[["question_en", "paragraph_de", "answer_en"]].copy()

df_2 = df_2.rename({"question_en": "Query", "paragraph_de": "Document", "answer_en": "Answer"}, axis=1)

df_2.to_csv("multilingual_data/mlqa_de-en.tsv", sep="\t", index=False)


df_2 = df[["question_de", "paragraph_de", "answer_de"]].copy()

df_2 = df_2.rename({"question_de": "Query", "paragraph_de": "Document", "answer_de": "Answer"}, axis=1)

df_2.to_csv("multilingual_data/mlqa_de-de.tsv", sep="\t", index=False)